# UEP-0239: Introduction to Geospatial Python

---

**A Tufts University Data Lab Tutorial**  
Written by [Uku-Kaspar Uustalu](https://directory.tufts.edu/user/view/90E8E773F8EC92B23679584546E5E321/)

Contact: <uku-kaspar.uustalu@tufts.edu>

Last updated: `GH_ACTIONS_DATE`

---

## Introduction to GeoPandas

[GeoPandas](https://geopandas.org/en/stable/) builds upon and extends [Pandas](https://pandas.pydata.org/docs/) to create a new [`geopandas.GeoDataFrame`](https://geopandas.org/en/stable/docs/reference/geodataframe.html) data structure that merges [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/frame.html) with [Shapely](https://shapely.readthedocs.io/en/stable/) geometric objects to create a user-friendly implementation of the [vector data model](https://saylordotorg.github.io/text_essentials-of-geographic-information-systems/s08-02-vector-data-models.html) suitable for use with geospatial data. The [`geopandas.GeoDataFrame`](https://geopandas.org/en/stable/docs/reference/geodataframe.html) data structure supports points, lines, and polygons, and keeps the spatial objects linked with their geospatial coordinate reference system (CRS) information and non-spatial attributes.

GeoPandas is usually imported under the alias `gpd`. Although GeoPandas uses Pandas is the background for non-spatial operations, Pandas still needs to be imported separately in order to use it directly.

In [ ]:
import pandas as pd
import geopandas as gpd

---

## Reading Data with Geographic Coordinates

There is a file named [`mbta-transit-stations.csv`](./data/mbta-transit-stations.csv) in the `data` directory. It is in [IETF RFC 4180 CSV](https://www.rfc-editor.org/info/rfc4180) (comma-separated values) format and contains information on the rapid transit stations of the Massachusetts Bay Transportation Authority (MBTA). Each row in the file represents a single unique rapid transit station with the following attributes:

- `stop_id` -- unique identifier for the station
- `stop_name` -- name of the station
- `stop_lat` -- latitude of the station in decimal degrees
- `stop_lon` -- longitude of the station in decimal degrees
- `stop_address` -- street address of the station
- `municipality` -- name of the municipality the station is within
- `accessible` -- whether the station is ADA accessible or not (`yes` or `no`)

Although this file contains geospatial information in the form of geographic coordinates, it is not in a geospatial data format. Hence it can be read in using [`pandas.read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) just like any other tabular CSV file.

In [ ]:
stops = pd.read_csv('data/mbta-transit-stations.csv')

In [ ]:
stops.head()

This [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/frame.html) can be converted into a [`geopandas.GeoDataFrame`](https://geopandas.org/en/stable/docs/reference/geodataframe.html) by passing it into the [`geopandas.GeoDataFrame()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html) constructor. The constructor takes two named arguments:

- `data` -- the attribute table for the vector data in the form of a [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/frame.html) (the non-spatial component of the data)
- `geometry` -- either an array of [Shapely](https://shapely.readthedocs.io/en/stable/) geometric objects (points, lines, or polygons) or the name of the column  in `data` containing these  objects (the spatial component of the data)

Although our data contains geographic coordinates in the form of latitude and longitude, it does not contain any vector objects in the form of points, lines, or polygons. However, the coordinates in the `stop_lat` and `stop_lon` columns do define points that could be used as the `geometry` of our GeoDataFrame. To convert these coordinates into an array of [`shapely.geometry.Point`](https://shapely.readthedocs.io/en/stable/manual.html#points) objects, we can use [`geopandas.points_from_xy()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html) and pass it the [`pandas.Series`](https://pandas.pydata.org/docs/reference/series.html) objects (DataFrame columns) that define the X (longitude) and Y (latitude) coordinates of the points.

In [ ]:
stops = gpd.GeoDataFrame(data=stops,
                         geometry=gpd.points_from_xy(x=stops.stop_lon,
                                                     y=stops.stop_lat))

In [ ]:
stops.head()

In [ ]:
type(stops)

In [ ]:
type(stops.geometry)

In [ ]:
type(stops.geometry[0])

---

## Coordinate Reference Systems

Note how `stops` is a [`geopandas.GeoDataFrame`](https://geopandas.org/en/stable/docs/reference/geodataframe.html) now and contains a new `geometry` column. This column is a [`geopandas.GeoSeries`](https://geopandas.org/en/stable/docs/reference/geoseries.html) object and it contains the spatial component of our data. In our case, this data is in the forms of points, which in the Python vector data model are  [`shapely.geometry.Point`](https://shapely.readthedocs.io/en/stable/reference/shapely.Point.html) objects.

The points themselves are just a collection of X and Y coordinates. The interpretation of those coordinates is defined by the [coordinate reference system](https://en.wikipedia.org/wiki/Spatial_reference_system) (CRS) of the data. While Shapely objects (the points) themselves are not coordinate-reference-system-aware, GeoPandas is and the CRS information of the data is stored in the GeoDataFrame itself. The CRS information can be accessed via the [`geopadnas.GeoDataFrame.crs`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.crs.html) attribute.

In [ ]:
stops.crs

Note how nothing is returned when we attempt to access the `crs` attribute of the `stops` GeoDataFrame. That is because we never defined a coordinate reference system for this dataset. Humans with a geospatial background could easily recognize that the longitude and latitude values in the data table are in decimal degrees and define the points' location on Earth in the  World Geodesic System (WGS) unprojected coordinate system. However, the computer does not know this and when we instructed GeoPandas to generate points from the latitude and longitude coordinates, we never provided any information on how to interpret the coordinates. We could have done this via the optional `crs` argument when calling [`geopandas.points_from_xy()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html).

Luckily we can still define the coordinate reference system of our GeoDataFrame using the [`geopandas.GeoDataFrame.set_crs()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_crs.html) method. This method works similarly to the *Define Projection* tool in desktop GIS software and should only be used when initially defining the CRS of the data. **Never use this method to reproject geospatial data!**

There are many ways of communicating to Python which CRS we would like to use, but the most robust method is to refer to the [European Petroleum Survey Group (EPSG) Geodetic Parameter Dataset](https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset) code of the coordinate reference system. The EPSG Geodesic Parameter Dataset is a public authoritative registry of coordinate reference systems and other related information where each CRS is assigned a unique EPSG code. This EPSG code can be used to reliably and precisely identify a coordinate reference system and is the preferred method of relaying CRS information in GeoPandas and other geospatial Python libraries.

The EPSG code of the WGS 84 unprojected coordinate reference system is [4326](https://epsg.org/crs_4326/index.html) and we can pass this number on to  [`geopandas.GeoDataFrame.set_crs()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_crs.html) via the `epsg` argument.

In [ ]:
stops = stops.set_crs(epsg=4326)

In [ ]:
stops.crs

In [ ]:
stops.crs.name

Note how now GeoPandas knows that this data is in the WGS 84 coordinate reference system. However, this data is still unprojected, meaning that any further analysis would need to take place on a spherical surface instead of a two-dimensional plain. This is computationally expensive and somewhat difficult to comprehend. Working with data in two-dimensional space is much more intuitive and and requires less complex mathematics.

Hence it is always good practice to transform (reproject) geospatial data into a local projected coordinate system before conducting any further analysis. Now that we have defined the CRS of our original data to be WGS 84, we can project the data into the Massachusetts State Plane projected coordinate system by having GeoPandas apply the appropriate CRS transformation. This can be done via the [`geopandas.GeoDataFrame.to_crs()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html) method, which is similar to the *Project* or *Reproject* tools in desktop GIS software. The EPSG code for the latest Massachusetts State Plane CRS is [26986](https://epsg.org/crs_26986/index.html) and we can pass this onto the method via the `epsg` argument. Additionally, we can specify `inplace=True` to apply the transformation to the original GeoDataFrame instead of returning a new one.

In [ ]:
stops.to_crs(epsg=26986, inplace=True)

In [ ]:
stops.head()

In [ ]:
stops.crs

In [ ]:
stops.crs.name

Note how the coordinates of the points in the `geometry` column of the GeoDataFrame have changed and GeoPandas now reports the CRS to be Massachusetts Mainland, which refers to the section of the projected Massachusetts State Plane CRS that is suitable for all of Massachusetts except Nantucket and Martha's Vineyard (which use the [EPSG:26987](https://epsg.org/crs_26987/index.html) Massachusetts Island CRS).

---

## Creating Static Maps

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as cx

In [ ]:
stops.geometry.x

In [ ]:
stops.geometry.y

In [ ]:
plt.scatter(stops.geometry.x, stops.geometry.y)
plt.show()


In [ ]:
stops.plot()
plt.show()

In [ ]:
ax = stops.plot(figsize=(10, 10), color='black', markersize=50)
cx.add_basemap(ax=ax, crs=stops.crs)
plt.show()

In [ ]:
ax = stops.plot(figsize=(10, 10), color='black', markersize=50)
cx.add_basemap(ax=ax, crs=stops.crs, source=cx.providers.OpenStreetMap.Mapnik)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

sns.kdeplot(x=stops.geometry.x,
            y=stops.geometry.y,
            fill=True,
            alpha=0.75,
            cmap='magma',
            ax=ax)

cx.add_basemap(ax=ax, crs=stops.crs)

plt.show()

In [ ]:
ax = stops[stops.accessible == 'yes'].plot(figsize=(10, 10),
                                           color='blue',
                                           label='accessible',
                                           marker='o',
                                           markersize=50)

stops[stops.accessible == 'no'].plot(color='red',
                                     label='not accessible',
                                     marker='x',
                                     markersize=100,
                                     ax=ax)

cx.add_basemap(ax=ax, crs=stops.crs)
plt.legend()
plt.show()

---

## Creating Interactive Maps

In [ ]:
import folium
import hvplot.pandas
import plotly.express as px

In [ ]:
stops.explore(column='accessible',
              tooltip='stop_name',
              tooltip_kwds=dict(labels=False),
              marker_kwds=dict(radius=5, fill=True),
              popup=True,
              legend=True,
              cmap='bwr_r',
              tiles='CartoDB positron')

In [ ]:
stops_wgs84 = stops.to_crs(epsg=4326)

In [ ]:
stops_wgs84_ctr = stops_wgs84.unary_union.centroid

In [ ]:
m = folium.Map(location=(stops_wgs84_ctr.y, stops_wgs84_ctr.x),
               zoom_start=12)

def style_function(feature):
    color = 'blue' if feature['properties']['accessible']=='yes' else 'red'
    return {'color': color, 'fillColor': color}

folium.GeoJson(data=stops_wgs84.to_json(),
               style_function = style_function,
               tooltip=folium.GeoJsonTooltip(fields=['stop_name'],
                                             labels=False),
               marker=folium.CircleMarker(radius=5,
                                          fill=True,
                                          fill_opacity=0.5)).add_to(m)

m

In [ ]:
stops.hvplot(crs=stops.crs.to_epsg(),
             tiles='StamenTerrainRetina',
             hover_cols=['stop_name'],
             c='accessible',
             cmap='bwr',
             alpha=0.7,
             size=200,
             width=1200,
             height=600)

In [ ]:
fig = px.scatter_mapbox(stops_wgs84,
                        lat=stops_wgs84.geometry.y,
                        lon=stops_wgs84.geometry.x,
                        color='accessible',
                        color_discrete_map={'yes': 'blue', 'no': 'red'},
                        hover_name='stop_name',
                        mapbox_style='carto-positron',
                        opacity=0.7,
                        zoom=10,
                        width=1200,
                        height=600)

fig.update_traces(marker={'size': 15})
fig.show()

---

## Exploring Shapely Objects

In [ ]:
from shapely.geometry import Polygon, LineString
from shapely.ops import transform
import pyproj

In [ ]:
stops[stops.stop_name == 'Medford/Tufts']

In [ ]:
stops[stops.stop_name == 'Medford/Tufts'].geometry

In [ ]:
medford = stops[stops.stop_name == 'Medford/Tufts'].geometry.values[0]

In [ ]:
medford

In [ ]:
type(medford)

In [ ]:
medford.x

In [ ]:
medford.y

In [ ]:
plt.scatter(medford.x, medford.y)
plt.show()

In [ ]:
boston = stops[stops.stop_name == 'Tufts Medical Center'].geometry.values[0]
smfa = stops[stops.stop_name == 'Museum of Fine Arts'].geometry.values[0]

In [ ]:
ax = stops.plot(figsize=(10, 10), color='black', markersize=25)
ax.scatter(medford.x, medford.y, color='blue', marker='*', s=200)
ax.scatter(boston.x, boston.y, color='red', marker='X', s=100)
ax.scatter(smfa.x, smfa.y, color='green', marker='s', s=50)
cx.add_basemap(ax, crs=stops.crs, source=cx.providers.CartoDB.Positron)
plt.show()

In [ ]:
triangle = Polygon([medford, boston, smfa])

In [ ]:
triangle

In [ ]:
type(triangle)

In [ ]:
triangle.exterior

In [ ]:
type(triangle.exterior)

In [ ]:
triangle.length

In [ ]:
triangle.exterior.length

In [ ]:
triangle.area

In [ ]:
triangle.exterior.area

In [ ]:
medford.distance(boston)

In [ ]:
medford.distance(smfa)

In [ ]:
triangle.exterior.xy

In [ ]:
x, y = triangle.exterior.xy
plt.plot(x, y)
plt.show()

In [ ]:
x, y = triangle.exterior.xy
plt.fill(x, y)
plt.show()

In [ ]:
x, y = triangle.exterior.xy
ax = stops.plot(figsize=(10, 10), color='black', markersize=50, zorder=3)
plt.plot(x, y, color='blue', linewidth=3, zorder=2)
plt.fill(x, y, color='blue', alpha=0.5, zorder=1)
cx.add_basemap(ax, crs=stops.crs, source=cx.providers.CartoDB.Positron)
plt.show()

In [ ]:
transformer = pyproj.Transformer.from_crs(stops.crs, 'epsg:4326').transform

In [ ]:
m = folium.Map(location=transform(transformer, triangle).centroid.coords[0],
               tiles='CartoDB positron',
               zoom_start=12)
folium.Polygon(locations=transform(transformer, triangle.exterior).coords,
                color='blue', fill=True).add_to(m)
folium.Marker(location=transform(transformer, medford).coords[0],
              tooltip='Medford/Tufts').add_to(m)
folium.Marker(location=transform(transformer, boston).coords[0],
              tooltip='Tufts Medical Center').add_to(m)
folium.Marker(location=transform(transformer, smfa).coords[0],
              tooltip='Museum of Fine Arts').add_to(m)
m

---

## Geocoding with GeoPy

In [ ]:
import random
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
glx_stops = ['Medford/Tufts', 'Ball Square', 'Magoun Square', 'Gilman Square',
             'East Somerville', 'Lechmere', 'Union Square']

In [ ]:
glx = pd.read_csv('data/mbta-transit-stations.csv')
glx = glx[glx.stop_name.isin(glx_stops)].reset_index(drop=True)
glx.drop(columns=['stop_lat', 'stop_lon', 'accessible'], inplace=True)

In [ ]:
glx

In [ ]:
token = 'uep239-python-spatial-{:04}'.format(random.randint(0,9999))
print(token)

In [ ]:
geolocator = Nominatim(user_agent=token)

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
glx['location'] = glx.stop_address.apply(geocode)

In [ ]:
glx

In [ ]:
glx['stop_lat'] = glx.location.apply(lambda loc: loc.latitude if loc else None)
glx['stop_lon'] = glx.location.apply(lambda loc: loc.longitude if loc else None)

In [ ]:
glx.drop(columns='location', inplace=True)

In [ ]:
glx

In [ ]:
glx = gpd.GeoDataFrame(data=glx,
                       geometry=gpd.points_from_xy(x=glx.stop_lon,
                                                   y=glx.stop_lat,
                                                   crs='epsg:4326'))
glx.to_crs(epsg=26986, inplace=True)

In [ ]:
ax = glx.plot(color='darkgreen', markersize=50)
cx.add_basemap(ax=ax, crs=glx.crs, source=cx.providers.CartoDB.Positron)
plt.show()

In [ ]:
glx_line = LineString(glx.set_index('stop_name').loc[glx_stops, 'geometry'])

In [ ]:
glx_line

In [ ]:
x, y = glx_line.xy
ax = glx.plot(color='darkgreen', markersize=50, zorder=2)
plt.plot(x, y, color='green', zorder=1)
cx.add_basemap(ax=ax, crs=glx.crs, source=cx.providers.CartoDB.Positron)
plt.show()

In [ ]:
m = folium.Map(location=transform(transformer, glx_line).centroid.coords[0],
               tiles='CartoDB positron',
               zoom_start=13)
folium.PolyLine(locations=transform(transformer, glx_line).coords,
                color='green').add_to(m)
folium.GeoJson(data=glx.to_crs(epsg=4326).to_json(),
               tooltip=folium.GeoJsonTooltip(fields=['stop_name'],
                                             labels=False),
               marker=folium.Marker(icon=folium.Icon(color='green',
                                                     icon='train-tram',
                                                     prefix='fa',))).add_to(m)
m

---

## Exercise

In [ ]:
entries = (pd.read_csv('data/mbta-gated-entries-2022.csv')
             .groupby('stop_id')
             .gated_entries.sum()
             .to_frame('total_entries')
             .reset_index())

In [ ]:
entries

In [ ]:
stops = stops.merge(entries, how='left', on='stop_id')

In [ ]:
stops

In [ ]:
ax = stops.plot(figsize=(10, 10),
                column='total_entries',
                cmap='Greens',
                alpha=0.8,
                edgecolor='darkgreen',
                legend=True,
                markersize=100,
                missing_kwds=dict(color='gray',
                                  edgecolor=None,
                                  alpha=0.5,
                                  marker='x',
                                  markersize=50))
cx.add_basemap(ax, crs=stops.crs, source=cx.providers.CartoDB.Positron)
plt.show()